In [14]:
import random
import numpy as np


1. Random bottle function


In [15]:
def pull():
    if random.random() < 0.1:
        return 9
    else:
        return 10


2. Theoretical expected value of one pull

\(E[X] = 10(0.9) + 9(0.1) = 9.9\) ounces.


In [16]:
expected_value = 10*0.9 + 9*0.1
expected_value


9.9

3. Simulation check of expected value (at least 100 pulls)


In [17]:
sim_avg = np.mean([pull() for _ in range(1000)])
sim_avg


np.float64(9.912)

NOW,Comparing the simulated average to the theoretical value 9.9.


4. Expected pulls until first 9 oz (with replacement)

Theoretically, for geometric distribution with \(p = 0.1\):  
\(E[\text{pulls}] = 1/p = 10\).


In [18]:
def pulls_until_short():
    count = 1
    while pull() == 10:
        count += 1
    return count

np.mean([pulls_until_short() for _ in range(5000)])


np.float64(9.977)

Compare this simulated average number of pulls with the theoretical value 10.


5. Real-world 10-case version (no replacement)

In reality, there are 10 different cases and exactly one is short.  
If we test each case once in random order, the expected posITion of the short case is:

\(\frac{1 + 2 + \cdots + 10}{10} = 5.5\) tests.

The code below simulates that by shuffling a list of 10 cases.


In [19]:
def find_short_case_no_replacement():
    cases = ["short"] + ["normal"]*9
    random.shuffle(cases)
    return cases.index("short") + 1

np.mean([find_short_case_no_replacement() for _ in range(5000)])


np.float64(5.4776)

We notice:

- With replacement: expected pulls ≈ 10  
- Realistic no-replacement 10-case test: expected tests ≈ 5.5  

This shows the original random-with-replacement method does not match the actual 10-case situation.  
The improved model uses a list (array) of cases, shuffled and tested once each.


Part 2: Plates




In [20]:
plates = ["P"]*6 + ["B"]*6 + ["K"]*6 + ["G"]*6
len(plates), plates.count("P")


(24, 6)

1. Probability of drawing a pink plate

There are 6 pink out of 24 plates:

\(P(\text{pink}) = 6/24 = 0.25\).


2. Probability that at least one of four plates is pink

We draw 4 plates without replacement.  
Compute:

\(P(\text{at least one pink}) = 1 - \dfrac{\binom{18}{4}}{\binom{24}{4}}\).


In [21]:
import math

def comb(n, r):
    return math.comb(n, r)

prob_at_least_one_pink = 1 - comb(18, 4)/comb(24, 4)
prob_at_least_one_pink


0.7120271033314511

Theoretical results:

- \(P(\text{single pink}) = 0.25\)  
- \(P(\text{at least one pink in 4}) \approx\) value computed above.


3. Simulation to test these probabilities


In [22]:
sim_p_single = np.mean([random.choice(plates) == "P" for _ in range(10000)])
sim_p_at_least_one = np.mean([("P" in random.sample(plates, 4)) for _ in range(10000)])
sim_p_single, sim_p_at_least_one


(np.float64(0.2478), np.float64(0.7109))

The simulated values should be close to the theoretical probabilities.


4. Stack model (LIFO) and dishwasher behavior

We represent the cabinet as a stack:
- Last plate in is the first plate out.  

We also keep a dishwasher list where dirty plates are collected, and when enough accumulate, we wash and shuffle them back on top of the stack.


In [23]:
def init_stack():
    s = plates[:]
    random.shuffle(s)
    return s

def wash(stack, dishwasher):
    random.shuffle(dishwasher)
    while dishwasher:
        stack.append(dishwasher.pop())


5. Compute probabilities (exactly one pink, at least one pink) with stack + washing

We simulate many dinners of 4 plates using the LIFO stack and washing behavior, without enforcing the daughter’s rejection rule yet.


In [24]:
def serve_family_dinner(stack, dishwasher):
    if len(stack) < 4:
        wash(stack, dishwasher)
    dinner = [stack.pop() for _ in range(4)]
    dishwasher.extend(dinner)
    if len(dishwasher) >= random.randint(10, 14):
        wash(stack, dishwasher)
    return dinner

def simulate_dinners_with_stack(n=10000):
    stack = init_stack()
    dishwasher = []
    count_exactly_one = 0
    count_at_least_one = 0

    for _ in range(n):
        dinner = serve_family_dinner(stack, dishwasher)
        pinks = dinner.count("P")
        if pinks == 1:
            count_exactly_one += 1
        if pinks >= 1:
            count_at_least_one += 1

    return count_exactly_one/n, count_at_least_one/n

simulate_dinners_with_stack()


(0.4858, 0.7282)

These simulated probabilities show how the stack and washing process affect the chances for exactly one pink and at least one pink.


6. Rejection process to always get at least one pink

Now we implement the rule:
- If the first 4 plates do not include a pink, we keep drawing until we find a pink.
- The extra non-pink plates drawn in that process are rejected and put back on top of the stack.


In [25]:
def draw_four_with_rejection(stack):
    if len(stack) < 4:
        return []

    drawn = [stack.pop() for _ in range(4)]
    if "P" in drawn:
        return drawn

    extra = []
    while stack:
        x = stack.pop()
        extra.append(x)
        if x == "P":
            break

    rejected = extra[:-1]
    while rejected:
        stack.append(rejected.pop())

    drawn[-1] = "P"
    return drawn


7. 1000-day simulation with rejection process and usage counts

We:
- Always ensure at least one pink using the rejection function.  
- Simulate 1000 days of dinners for 4 people.  
- Track how often each color is used.  
- Examine how the stack evolves.


In [26]:
from collections import Counter

def simulate_1000_days():
    stack = init_stack()
    dishwasher = []
    usage = Counter()

    for _ in range(1000):
        if len(stack) < 4:
            wash(stack, dishwasher)

        dinner = draw_four_with_rejection(stack)
        if not dinner:
            break

        usage.update(dinner)
        dishwasher.extend(dinner)

        if len(dishwasher) >= 12:
            wash(stack, dishwasher)

    return stack, usage

final_stack, usage_counts = simulate_1000_days()
usage_counts, final_stack[:10]


(Counter({'P': 2510, 'K': 658, 'G': 776, 'B': 56}),
 ['P', 'P', 'G', 'P', 'P', 'P', 'K', 'P'])

Interpretation of results:

- Pink plates are used more frequently than other colors (higher usage counts).  
- Pink plates tend to migrate toward the top of the stack.  
- Some non-pink plates may have low usage, staying near the bottom and rarely being drawn.  

This shows how always satisfying the daughter with at least one pink plate biases the usage toward pink plates over many days.
